In [1]:
from app.variable.expr import LazyExpr
from typing import List, Dict, Any, Optional
import ast


In [2]:
# im = ["from datetime import datetime", "from typing import List"]
im = []

# s = "lambda x: all(isinstance(i, datetime) for i in x) if isinstance(x, list) else False"
s = "(aaaaa := a + 1)"
# s = """
# 1+1
# """
f = LazyExpr(s, im)

# print(ast.dump(f._ast, indent=2))
print(f.unbound)
print(f.target)

f.eval(env={"a": 1024})

{'a'}
aaaaa


1025

In [3]:
from app.variable.model import Variable, LiteralVariable

lst = [
    LiteralVariable("s", LazyExpr("1+2")),
    LiteralVariable("t", LazyExpr("s*2+12")),
    LiteralVariable("a", LazyExpr("1+1")),
    LiteralVariable("b", LazyExpr("a+1")),
    LiteralVariable("c", LazyExpr("b+1")),
    LiteralVariable("d", LazyExpr("a+b+2")),
    LiteralVariable("e", LazyExpr("c+d+3")),
]
for v in lst:
    print(v.unbound)

set()
{'s'}
set()
{'a'}
{'b'}
{'b', 'a'}
{'c', 'd'}


In [4]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()
for v in lst:
    G.add_node(v.name)
    for u in v.unbound:
        G.add_edge(v.name, u)

In [5]:
# nx.draw(G, with_labels=True)
# plt.show()

In [6]:
valid_names = list(map(lambda x: x.name, lst))
invalid_names = list(filter(lambda x: x not in valid_names, G.nodes))
if invalid_names:
    raise ValueError(f"Invalid names: {invalid_names}") # unresolved names
else:
    print("All names are valid")

topo: list[str] = []
try:
    topo = list(nx.topological_sort(G))
    print(f"Topological sort: {topo}")
except nx.NetworkXUnfeasible:
    raise ValueError("Cycle detected")

All names are valid
Topological sort: ['t', 'e', 's', 'c', 'd', 'b', 'a']


In [7]:
# the evaluation order is reversed

env = {}
def sort_by_topo(lst, topo):
    # Create a mapping from variable names to their positions in topo
    index_map = {name: i for i, name in enumerate(reversed(topo))}
    # Sort the list based on the mapping
    sorted_lst = sorted(lst, key=lambda var: index_map[var.name])
    return sorted_lst

sorted_lst:List[LiteralVariable] = sort_by_topo(lst, topo)

env = {}
for v in sorted_lst:
    env[v.name] = v.value(env)
    print(f"{v.name} = {env[v.name]}")

print(env)

a = 2
b = 3
d = 7
c = 4
s = 3
e = 14
t = 18
{'a': 2, 'b': 3, 'd': 7, 'c': 4, 's': 3, 'e': 14, 't': 18}


In [8]:
from app.dependency.graph import DependencyResolver

resolver = DependencyResolver()
resolver.add_many(lst)
print(resolver.env)

Ok({'a': 2, 'b': 3, 'd': 7, 'c': 4, 's': 3, 'e': 14, 't': 18})


In [9]:
text = "本期中出院病例${num_patient}例。手术人数${num_operation}人，四级手术${num_operation_4}人，微创手术${num_operation_micro}人。CMI值${CMI}，其中 CMI 小于1的病人数有${query(CMI<1)}人，占比${query(CMI<1)/num_patient}；1-2的病人数${query(CMI > 1 and CMI < 2)}人，占比${query(CMI > 1 and CMI < 2) / num_patient};2-5的病人数${(num_CIM_2_5 := query(CMI > 2 and CMI < 5))}人，占比${num_CIM_2_5 / num_patient};大于5的病人数${(num_CIM_gt_5 := query(CMI > 5))}人,占比${num_CIM_gt_5 / num_patient}。总体来说大于2的病人数${(num_CIM_gt_2 := query(CMI > 2))}人，占比${num_CIM_gt_2 / num_patient}%，有提升空间。"

In [10]:
from app.dependency.extractor import parse

parsed = parse(text)

# print(parsed)

table = parsed.unwrap().table
exprs = list(map(lambda x: LazyExpr(x[1]), table))
print(list(map(lambda x: x.raw, exprs)))
print(list(map(lambda x: x.target, exprs)))
print(list(map(lambda x: x.unbound, exprs)))

['num_patient', 'num_operation', 'num_operation_4', 'num_operation_micro', 'CMI', 'query(CMI<1)', 'query(CMI<1)/num_patient', 'query(CMI > 1 and CMI < 2)', 'query(CMI > 1 and CMI < 2) / num_patient', '(num_CIM_2_5 := query(CMI > 2 and CMI < 5))', 'num_CIM_2_5 / num_patient', '(num_CIM_gt_5 := query(CMI > 5))', 'num_CIM_gt_5 / num_patient', '(num_CIM_gt_2 := query(CMI > 2))', 'num_CIM_gt_2 / num_patient']
[None, None, None, None, None, None, None, None, None, 'num_CIM_2_5', None, 'num_CIM_gt_5', None, 'num_CIM_gt_2', None]
[{'num_patient'}, {'num_operation'}, {'num_operation_4'}, {'num_operation_micro'}, {'CMI'}, {'CMI', 'query'}, {'CMI', 'num_patient', 'query'}, {'CMI', 'query'}, {'CMI', 'num_patient', 'query'}, {'CMI', 'query'}, {'num_CIM_2_5', 'num_patient'}, {'CMI', 'query'}, {'num_CIM_gt_5', 'num_patient'}, {'CMI', 'query'}, {'num_CIM_gt_2', 'num_patient'}]
